##### Translating data

In [ ]:
import os

translate = {"cane": "dog", "cavallo": "horse", "elefante": "elephant", "farfalla": "butterfly", "gallina": "chicken", "gatto": "cat", "mucca": "cow", "pecora": "sheep", "scoiattolo": "squirrel", "dog": "cane", "cavallo": "horse", "elephant" : "elefante", "butterfly": "farfalla", "chicken": "gallina", "cat": "gatto", "cow": "mucca", "spider": "ragno", "squirrel": "scoiattolo"}

def rename_folders(path):
  for folder in os.listdir(path):
    if os.path.isdir(os.path.join(path, folder)):
      translated_name = translate.get(folder, folder)
      new_path = os.path.join(path, translated_name)
      os.rename(os.path.join(path, folder), new_path)
      print(f'Renamed folder: {folder} to {translated_name}')

rename_folders('raw-img')

In [ ]:
%pip install keras
%pip install tensorflow
%pip install pillow
%pip install scipy

In [1]:
from keras import models
from keras import layers
from keras import activations
from keras import initializers
from keras import regularizers
from keras import optimizers
from keras import losses
from keras import metrics
from keras import callbacks
from keras.preprocessing import image

In [8]:
model = models.Sequential()

model.add(layers.Conv2D(
  32, (5, 5), # 32: numero de filtros e (5,5) é o shape do kernel que vai ser passado pela imagem, ela perde n-1 do tamanho em cada dimensão
  input_shape = (64, 64, 3), # 64: tamanho da imagem, 3: diz que a imagem é RGB, se for preto e branco é 1
  activation = 'relu'
))

model.add(layers.MaxPooling2D(
  pool_size = (2, 2) # reduz o tamanho da imagem pela metade, passa um kernel 2x2 e retorna um valor 1x1
))

model.add(layers.Conv2D(
  16, (5, 5),
  input_shape = (30, 30, 3),  # 64 - 4 = 60 - 30 do MaxPooling = 30, 30
  activation = 'relu'
))

model.add(layers.MaxPooling2D(
  pool_size = (2, 2)
))

# model.add(layers.Conv2D(
#   4, (5, 5),
#   input_shape = (13, 13, 3),
#   activation = 'relu'
# ))

# model.add(layers.MaxPooling2D(
#   pool_size = (2, 2)
# ))

model.add(layers.Flatten()) # transforma a imagem de matriz 2d para vetor

model.add(layers.Dense(512, # quantidade de neurônios
  # kernel_regularizer = regularizers.L2(1e-4), # penaliza pesos altos
  # kernel_initializer = initializers.RandomNormal(stddev=1), # inicialização dos pesos, inicia com desvio-padrão de 1, tem valores mais naturais
  # bias_initializer = initializers.Zeros() # inicia o bias como 0
))

model.add(layers.Activation(activations.relu)) # filtra se o neurônio é importante (relu mantem os positivos e zera os negativos)

# model.add(layers.BatchNormalization()) # normaliza os valores de saída da camada para reduzir overfitting, melhorar desempenho.

# model.add(layers.Activation(activations.relu))

model.add(layers.Dropout(0.2)) # diz que 20% dos neurônios serão desligados para evitar overfitting

model.add(layers.Dense(10,
  # kernel_regularizer = regularizers.L2(1e-4), 
  # kernel_initializer = initializers.RandomNormal(stddev=1),
  # bias_initializer = initializers.Zeros() 
))

model.add(layers.Activation(activations.softmax))

model.compile(
  optimizer=optimizers.Adam(), # Adam e SGD são os optimizers mais utilizados
  loss=losses.CategoricalCrossentropy(),
  metrics=[metrics.Accuracy(), metrics.Precision()]
)

dataGen = image.ImageDataGenerator(
  rescale = 1.0/255, # redimensiona a escala de 0 a 255 para 0 a 1
  shear_range = 0.2,
  zoom_range = 0.2,
  horizontal_flip = True,
  vertical_flip = False,
  validation_split = 0.2
)

X_train = dataGen.flow_from_directory(
  'raw-img',
  target_size = (64, 64), # tamanho desejado da imagem, redimensiona.
  batch_size = 32, # passa a quantidade de imagens desejadas para treino
  class_mode = 'categorical', # binarico, categorico e de regressão.
  subset = 'training'
)

X_test = dataGen.flow_from_directory(
  'raw-img',
  target_size = (64, 64), # tamanho desejado da imagem, redimensiona.
  batch_size = 32, # passa a quantidade de imagens desejadas para treino
  class_mode = 'categorical', # binarico, categorico e de regressão.
  subset = 'validation'
)

total_data = 20947
batch_size = 32

model.fit(
  X_train, 
  validation_data=X_test,
  steps_per_epoch = total_data / batch_size, # steps_per_epoch * batch_size == len(dataset) <- recomendado
  epochs = 50,
  validation_steps = (total_data / batch_size) / 10, # steps_per_epoch / 10 <- recomendado
  callbacks = [
    callbacks.ModelCheckpoint(
      filepath = 'trained/model.{epoch:02d}-{val_loss:.2f}.h5'
    )
  ]
) 

model.save('model')

Found 20947 images belonging to 10 classes.
Found 5232 images belonging to 10 classes.
Epoch 1/50
654/654 [==============================] - 41s 61ms/step - loss: 1.9506 - accuracy: 0.0000e+00 - precision_6: 0.6187 - val_loss: 1.7327 - val_accuracy: 0.0000e+00 - val_precision_6: 0.6214
Epoch 2/50
654/654 [==============================] - 40s 61ms/step - loss: 1.6281 - accuracy: 0.0000e+00 - precision_6: 0.6824 - val_loss: 1.5124 - val_accuracy: 0.0000e+00 - val_precision_6: 0.7238
Epoch 3/50
654/654 [==============================] - 40s 61ms/step - loss: 1.4876 - accuracy: 0.0000e+00 - precision_6: 0.7093 - val_loss: 1.4479 - val_accuracy: 0.0000e+00 - val_precision_6: 0.7019
Epoch 4/50
654/654 [==============================] - 40s 61ms/step - loss: 1.3996 - accuracy: 0.0000e+00 - precision_6: 0.7213 - val_loss: 1.3827 - val_accuracy: 0.0000e+00 - val_precision_6: 0.7278
Epoch 5/50
654/654 [==============================] - 40s 60ms/step - loss: 1.3184 - accuracy: 0.0000e+00 - preci

KeyboardInterrupt: 